In [ ]:
import numba
import numpy as np
from typing import Union
from numba import njit, prange
from numba.np.numpy_support import from_dtype, as_dtype
from numba.types import uint32, int64, float64, bool_
from numba.experimental import jitclass


@jitclass
class RingBufferF642D:
    """
    A fixed-size circular buffer using Numba JIT compilation.

    Can only support F64 values.

    Parameters
    ----------
    len_1dim : int
        The length of 1D arrays you intend to add.

    len_2dim : int
        The capacity of the ringbuffer (number of 1D arrays it will hold).
    """

    _len_1dim_: uint32
    _len_2dim_: uint32
    _left_index_: uint32
    _right_index_: uint32
    _is_empty_: bool_
    _arr_: float64[:]

    def __init__(self, len_1dim: int, len_2dim: int) -> None:
        self._len_1dim_ = len_1dim
        self._len_2dim_ = len_2dim
        self._left_index_ = 0
        self._right_index_ = 0
        self._is_empty_ = True
        self._arr_ = np.zeros(shape=self._len_1dim_ * self._len_2dim_, dtype=float64)

    def __len__(self) -> int:
        return (self._right_index_ - self._left_index_) // self._len_1dim_

    def __getitem__(self, item: int) -> np.ndarray:
        real_start_idx = item * self._len_1dim_
        real_end_idx = real_start_idx + self._len_1dim_
        return self.as_array()[real_start_idx : real_end_idx]

    @property
    def dtype(self) -> np.dtype:
        """Data type of the array's contents"""
        return self._arr_.dtype

    @property
    def is_full(self) -> bool:
        """True if there is no more empty space in the buffer"""
        return (self._right_index_ - self._left_index_) == self._len_2dim_
    
    def as_array(self) -> np.ndarray:
        """Returns a linearized form of the buffer's contents."""
        part1 = self._arr_[self._left_index_ : min(self._right_index_, self._len_2dim_)]
        part2 = self._arr_[: max(self._right_index_ - self._len_2dim_, 0)]
    
        unwrapped_array = np.empty_like(self._array_)
        unwrapped_array[:part1.size] = part1
        unwrapped_array[part1.size:] = part2
        return unwrapped_array.reshape()

    def _fix_indices_(self) -> None:
        """Corrects the indices if they exceed the buffer's capacity."""
        if self._left_index_ >= self._len_2dim_:
            self._left_index_ -= self._len_2dim_
            self._right_index_ -= self._len_2dim_
        elif self._left_index_ < 0:
            self._left_index_ += self._len_2dim_
            self._right_index_ += self._len_2dim_

    def appendright(self, values: np.ndarray[np.float64]) -> None:
        """Adds an element to the end of the buffer."""
        assert values.size == self._len_1dim_
        for value in values:
            if self.is_full:
                self._left_index_ += 1

            self._arr_[self._right_index_ % self.capacity] = value
            self._right_index_ += 1
            self._fix_indices_()

    def appendleft(self, values: np.ndarray[np.float64]) -> None:
        """Adds an element to the start of the buffer."""
        for value in values:
            if self.is_full:
                self._right_index_ -= 1

            self._left_index_ -= 1
            self._fix_indices_()
            self._arr_[self._left_index_] = value

    def popright(self) -> np.ndarray[np.float64]:
        """Removes and returns an element from the end of the buffer."""
        if len(self) == 0:
            raise IndexError("Cannot pop from an empty RingBuffer")

        self._right_index_ -= 1
        self._fix_indices_()
        res = self._arr_[self._right_index_ % self.capacity]
        return res

    def popleft(self) -> np.ndarray[np.float64]:
        """Removes and returns an element from the start of the buffer."""
        if len(self) == 0:
            raise IndexError("Cannot pop from an empty RingBuffer")

        res = self._arr_[self._left_index_]
        self._left_index_ += 1
        self._fix_indices_()
        return res

    def reset(self) -> np.ndarray:
        """Clears the buffer and resets it to its initial state."""
        res = self.as_array()
        self._arr_.fill(0.0)
        self._left_index_ = 0
        self._right_index_ = 0
        return res